In [ ]:
# Hack the sys path for import
import sys
sys.path.insert(0,'..')

In [ ]:
import numpy as np
import pandas as pd

import scipy.stats as stats

from faker import Faker
import datetime as dt
from datetime import date

from database import Database

from pandas2sql.generators import SQLiteGenerator, MySQLGenerator

## Setup

In [ ]:
db = Database("oefeningen.db")
gen = SQLiteGenerator()

## Transacties

In [ ]:
def fake_dates(dt_start, dt_end, n):
    fk = Faker("nl_NL")
    dt_start = dt.datetime.strptime(dt_start, "%Y-%m-%d")
    dt_end = dt.datetime.strptime(dt_end, "%Y-%m-%d")
    return [fk.date_between_dates(dt_start, dt_end) for _ in range(n)]

In [ ]:
n = 150
n_products = 20
n_customers = 30
date_range = "2020-01-01", "2020-03-31"

transactions = pd.DataFrame({
    "Id": np.arange(0, n),
    "ProductId": np.random.randint(0, n_products, n),
    "KlantId": np.random.randint(0, n_customers, n),
    "Aantal": stats.halfnorm(1, 3).rvs(n).astype(int),
    "PrijsExclusief": [round(abs(x), 2) for x in stats.norm(5, 2).rvs(n)],
    "BTWTarief": np.random.choice(["hoog", "laag"], n),
    "BestelDatum": fake_dates(date_range[0], date_range[1], n),
}).assign(
    LeverDatum=lambda df: df["BestelDatum"] + [dt.timedelta(days=int(x)) for x in np.random.randint(1, 5, n)]
)

In [ ]:
db.query("DROP TABLE IF EXISTS Transacties;")
db.query(gen.generate_schema(transactions, "Transacties"))

In [ ]:
for insert in gen.generate_inserts(transactions, "Transacties"):
    print(db.query(insert))

In [ ]:
db.query("SELECT * FROM Transacties LIMIT 5;")

## Koppelingen

In [ ]:
db.query("DROP TABLE IF EXISTS KoppelA;")
db.query("DROP TABLE IF EXISTS KoppelB;")

db.query("""
    CREATE TABLE KoppelA (
        Id INTEGER,
        Tekst TEXT
    );
""")

db.query("""
    INSERT INTO KoppelA (Id, Tekst) 
    VALUES
        (1, 'ID 1 in A'),
        (2, 'ID 2 in A'),
        (4, 'ID 4 in A'),
        (4, 'ID 4 duplicaat in A'),
        (5, 'ID 5 in A'),
        (6, 'ID 6 in A'),
        (6, 'ID 6 duplicaat in A')
    ;
""")

db.query("""
    CREATE TABLE KoppelB (
        Id INTEGER,
        Tekst TEXT
    );
""")


db.query("""
INSERT INTO KoppelB (Id, Tekst) 
VALUES
    (1, 'ID 1 in B'),
    (3, 'ID 3 in B'),
    (4, 'ID 4 in B'),
    (5, 'ID 5 in B'),
    (5, 'ID 5 duplicaat in B'),
    (6, 'ID 6 in B'),
    (6, 'ID 6 duplicaat in B')
;
""")

## MySQL

In [ ]:
mysql_gen = MySQLGenerator()

In [ ]:
print(mysql_gen.generate_schema(transactions, "Transacties"))

In [ ]:
for insert in mysql_gen.generate_inserts(transactions, "Transacties"):
    print(insert)